<a href="https://colab.research.google.com/github/harrywinks/2019_cau_oss_hackathon/blob/master/hackathon_team02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 초기 환경 설정**



In [10]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

is_mnist = True;

# 데이터셋 로드
# x_train, y_train: 트레이닝 데이터 및 레이블
# x_test, y_test: 테스트 데이터 및 레이블
if is_mnist:
  data_type = 'mnist'
  (x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() # fashion MNIST 데이터셋인 경우,
else:
  data_type = 'cifar10'
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data() # cifar10 데이터셋인 경우,


# 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 총 클래스 개수
num_classes = y_test.shape[1]

# 인풋 데이터 타입
input_shape = x_test.shape[1:]

4423680/4422102 [==============================] - 0s 0us/step


# **2. 데이터 전처리**


In [0]:
from matplotlib import pyplot
from keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, DepthwiseConv2D, SeparableConv2D
from tensorflow.keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D
from tensorflow.keras.layers import BatchNormalization, concatenate, add, Dropout, ReLU, Lambda, Activation, LeakyReLU

In [0]:
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0
if is_mnist:
  x_train_after=x_train_after.reshape(x_train_after.shape[0],28,28,1)
  x_test_after=x_test_after.reshape(x_test_after.shape[0],28,28,1)
  input_shape=x_test_after.shape[1:]

In [13]:
#실시간 데이터 확대로 텐서 이미지 데이터의 배치를 생성합니다. 데이터가 일괄 처리됩니다.
from keras.preprocessing.image import ImageDataGenerator  
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True)

datagen.fit(x_train_after)

type(x_train_after)

numpy.ndarray

# **3. 모델 생성**



In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import multi_gpu_model
#keras와 tensorflow.keras가 섞여서 집어넣은 코드
import keras.backend as K
from keras.callbacks import ModelCheckpoint
import os

In [15]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 monitor='val_acc',
                                                 save_best_only=True, 
                                                 save_weights_only=True,
                                                 mode = 'max', 
                                                 verbose=0,
                                                 period=1)

W0824 08:50:44.162117 140623657674624 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [16]:


import keras.backend as K
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
batch_size = 128
num_classes = 10



droprate=0.25
model = keras.Sequential()
#convolution 1st layers
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding="same",
                 activation='relu',
                 input_shape=input_shape)) #0
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(droprate))#3
#model.add(MaxPooling2D())

#convolution 2nd layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))#1
model.add(BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Dropout(droprate))#3

#convolution 3rd layers
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))#1
model.add(BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D())
model.add(tf.keras.layers.Dropout(droprate))#3

#Fully connected 1st layers
model.add(tf.keras.layers.Flatten()) #7
model.add(tf.keras.layers.Dense(500,use_bias=False)) #13
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu')) #14
model.add(tf.keras.layers.Dropout(droprate))      #15

#Fully connected final layer
model.add(tf.keras.layers.Dense(10)) #8f
model.add(tf.keras.layers.Activation('softmax')) #9

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.RMSprop(), metrics=['accuracy'])

model.fit(x_train_after, y_train, batch_size = 128,epochs = 15, shuffle=True,
          validation_data=[x_test_after, y_test],callbacks = [cp_callback])

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 11s 188us/sample - loss: 0.4330 - acc: 0.8428 - val_loss: 0.7016 - val_acc: 0.7729
Epoch 2/15
60000/60000 [==============================] - 10s 172us/sample - loss: 0.2888 - acc: 0.8943 - val_loss: 0.2862 - val_acc: 0.8968
Epoch 3/15
60000/60000 [==============================] - 10s 172us/sample - loss: 0.2514 - acc: 0.9079 - val_loss: 0.2700 - val_acc: 0.9021
Epoch 4/15
60000/60000 [==============================] - 10s 172us/sample - loss: 0.2259 - acc: 0.9176 - val_loss: 0.2517 - val_acc: 0.9084
Epoch 5/15
60000/60000 [==============================] - 10s 172us/sample - loss: 0.2126 - acc: 0.9231 - val_loss: 0.2300 - val_acc: 0.9176
Epoch 6/15
60000/60000 [==============================] - 10s 173us/sample - loss: 0.1949 - acc: 0.9278 - val_loss: 0.2424 - val_acc: 0.9151
Epoch 7/15
60000/60000 [==============================] - 10s 172us/sample - loss: 0.1861 - acc: 0.9333 

# **4. 모델 저장**

In [0]:
save_path = '/content/'
team_name = 'team02'

# 모델의 weight 값만 저장합니다.
model.save_weights(save_path + 'model_weight_' + data_type + '_' + team_name + '.h5')

# 모델의 구조만을 저장합니다.
model_json = model.to_json()
with open(save_path + 'model_structure_' + data_type + '_' + team_name + '.json', 'w') as json_file : 
    json_file.write(model_json)

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_' + data_type + '_' + team_name + '.h5')

# **5. 모델 로드 및 평가**

In [18]:
save_path = '/content/'
team_name = 'team02'

model = keras.models.load_model(save_path + 'model_entire_' + data_type + '_' + team_name + '.h5')
model.evaluate(x_test_after, y_test)

10000/10000 [==============================] - 1s 112us/sample - loss: 0.2177 - acc: 0.9319


[0.21768041597157717, 0.9319]